In [ ]:
# F1 score
from sklearn.metrics import f1_score
#[1,2,3,2,1,3]
#[1,2,3,1,1,3]
def cal_macro_f1(y_true,y_pred):
    score = f1_score(y_true,y_pred,average='macro')
    return score

from sklearn.model_selection import train_test_split

X_train, X_eval, y_train1, y_eval = train_test_split(x_train, y_train, test_size=0.2, random_state=2019)

# Cross Validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=666)
train_matrix = np.zeros((df_train.shape[0],2)) #prob for evaluation
test_pre_matrix = np.zeros((10,df_test.shape[0],2)) #save test prob
cv_scores=[] #cv scores
acc_scores=[]
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
for i,(train_index,eval_index) in enumerate(kf.split(x_train)):
    print(len(train_index),len(eval_index))
    
    
    #training set
    X_train = x_train.iloc[train_index,:]
    # y_train1 = df_train.iloc[train_index,-1]
    y_train1 = y_train[train_index]
    
    #validation set
    X_eval = x_train.iloc[eval_index,:]
    y_eval = y_train[eval_index]
    

    model =lgb.LGBMClassifier(boosting_type='gbdt', 
                   num_leaves=2**10,
                   max_depth=-1, 
                   learning_rate= 0.01,
                   n_estimators=500,
                   objective='binary',
                   subsample=0.7,#
                   colsample_bytree=0.5,#
                   reg_lambda=3,#l2
                   n_jobs=16, #
                   silent=True, 
                   random_state=2019,
                #    colsample_bylevel=0.5,
                   subsample_for_bin=300000,
                   min_child_weight=1.5
                #    metric='multi_logloss'
                  )
    model.fit(X_train,y_train1,eval_set=(X_eval,y_eval), early_stopping_rounds=20)
    ####predict for validation set
    eval_prob = model.predict_proba(X_eval)
    eval_pred = np.argmax(eval_prob,axis=1)
    score = cal_macro_f1(y_eval,eval_pred)
    acc = sum(eval_pred == y_eval) / len(y_eval)

    acc_scores.append(acc)
    cv_scores.append(score)
    print("validation F1 score is",score)
    print("validation Accuracy score is",acc)